In [22]:
# insert your credentials here
import pandas as pd
import os
from dotenv import load_dotenv
import dataset
import json
import sys
import tweepy
from sqlalchemy.exc import ProgrammingError
import pandas as pd

In [23]:
load_dotenv()

db = dataset.connect(os.getenv("DATABASE_URL"))

# Variables that contains the credentials to access Twitter API
CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_KEY = os.getenv('ACCESS_KEY')
ACCESS_SECRET = os.getenv('ACCESS_SECRET')

In [24]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

# initialize Tweepy API
api = tweepy.API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

In [25]:
class StreamListener(tweepy.StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(StreamListener,self).__init__()
        self.output_file = output_file
        self.counter = 0

    def on_status(self, status):
        self.counter = self.counter + 1
        conditions = (not 'RT @' in status.text)
        if conditions:
            description = status.user.description
            loc = status.user.location
            text = status.text
            coords = status.coordinates
            geo = status.geo
            name = status.user.screen_name
            user_created = status.user.created_at
            id_str = status.id_str
            created = status.created_at
            source = status.user.url
            language = status.lang

            if geo is not None:
                geo = json.dumps(geo)

            if coords is not None:
                coords = json.dumps(coords)

            table = db["tweets"]
            try:
                table.insert(dict(
                    user_description=description,
                    user_location=loc,
                    coordinates=coords,
                    text=text,
                    geo=geo,
                    user_name=name,
                    user_created=user_created,
                    id_str=id_str,
                    created=created,
                    source = source,
                    language = language,
                    ))
            except ProgrammingError as err:
                print(err)

    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        if status_code == 420:
            #return False in on_data disconnects the stream
            return False

    # When a deleted tweet appears
    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return

    # When reach the rate limit
    def on_limit(self, track):
        print("Rate limited, continuing")
        # Continue mining tweets
        return True

In [ ]:
# create instance of the tweepy tweet stream listener
stream_listener = StreamListener()

# create instance of the tweepy stream
stream = tweepy.Stream(auth=auth, listener=stream_listener, tweet_mode="extended")

# words to search for
track = ["police", "cop", "officer"]

# search twitter for programming languages
stream.filter(track=track, languages = ['en', 'und'])

In [ ]:
df_raw = pd.DataFrame(db['tweets'])
df = df_raw[["id_str", "text"]]
df.rename(columns={'id_str': 'ids'}, inplace=True)
df